In [1]:
import sys
import os
import pandas as pd
import warnings
import json
from devtools import pprint

# # Add the src folder to the path
# sys.path.append(os.path.abspath('../..'))
# warnings.filterwarnings("ignore", category=FutureWarning)

# Import the classes
from utils.polygon_utils import polygon, prepare_news_for_llm, parse_news_by_date
from utils.data_models import NewsSentiment, NewsSentimentByDate
from utils.connect_db import ConnectDB
from utils.llm.prompt import prompts as p
from utils.llm.api_call import call_llm
from langchain_core.prompts import ChatPromptTemplate

In [2]:
# Reload modules to pick up changes
import importlib
import utils.polygon_utils
import utils.llm.prompt
import utils.llm.api_call
importlib.reload(utils.polygon_utils)
importlib.reload(utils.llm.prompt)
importlib.reload(utils.llm.api_call)
from utils.polygon_utils import polygon, prepare_news_for_llm, parse_news_by_date
from utils.llm.prompt import prompts as p
from utils.llm.api_call import call_llm

In [3]:
pl = polygon()

POLYGON_API_KEY is set
DB_USER and DB_PASSWORD are set


In [8]:
ticker = 'AAPL'
start_date = '2026-01-05'
end_date = '2026-01-14'

In [9]:
nvdia_news, nvdia_news_df = pl.get_news(ticker, start_date, end_date, limit=1000, strict=False)

(Background on this error at: https://sqlalche.me/e/20/e3q8)
Error inserting company news data for AAPL: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'jsedocc7.scrc.nyu.edu' (timed out)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [10]:
nvdia_news_df.sort_index(ascending=False)

,article_url,author,description,polygon_id,insights,keywords,published_utc,tickers,title,publisher,ticker
79,https://www.fool.com/investing/2026/01/14/warr...,Anthony Di Pizio,Warren Buffett stepped down as CEO of Berkshir...,043959b9342841ebb3e877dc2b389af972d064b8abddb4...,"[{""ticker"": ""BRK.A"", ""sentiment"": ""positive"", ...","[""Warren Buffett"", ""index funds"", ""S&P 500"", ""...",2026-01-14 11:30:00,"[""BRK.A"", ""BRK.B"", ""VOO"", ""NVDA"", ""MSFT"", ""AAP...","Warren Buffett Stepped Down, but His Timeless ...",The Motley Fool,AAPL
78,https://www.benzinga.com/markets/tech/26/01/49...,Namrata Sen,President Trump claimed credit for Intel's sub...,786b33d7b8c83e3cd84bbd775b99163bfb8c23e72f6224...,"[{""ticker"": ""INTC"", ""sentiment"": ""positive"", ""...","[""Intel investment"", ""Trump administration"", ""...",2026-01-14 09:01:39,"[""INTC"", ""AAPL"", ""NVDA"", ""SFTBY""]",Trump Takes Credit For Intel's Surge But Hesit...,Benzinga,AAPL
77,https://www.fool.com/coverage/etfs/2026/01/14/...,Josh Kohn-Lindquist,Vanguard's VOOG and Invesco's RSP offer differ...,ffe535bc14d5ce88b071de2518e83cbe1137183283c07a...,"[{""ticker"": ""NVDA"", ""sentiment"": ""neutral"", ""s...","[""S&P 500 ETF comparison"", ""VOOG vs RSP"", ""gro...",2026-01-14 06:27:09,"[""NVDA"", ""AAPL"", ""MSFT""]",Which Variation of the S&P 500 Is Better: Vang...,The Motley Fool,AAPL
76,https://www.fool.com/investing/2026/01/14/big-...,Motley Fool Staff,CES 2026 showcased major announcements in auto...,9bcffaef727c2469199e362ac2340a94de35ebb580b938...,"[{""ticker"": ""UBER"", ""sentiment"": ""positive"", ""...","[""CES 2026"", ""autonomous vehicles"", ""robotaxi""...",2026-01-14 05:13:00,"[""UBER"", ""LCID"", ""NVDA"", ""META"", ""AAPL"", ""GM"",...",Big Investing Takeaways From CES 2026,The Motley Fool,AAPL
75,https://www.fool.com/coverage/stock-market-tod...,Emma Newbery,"On January 13, 2026, the stock market declined...",76640b1b81aaaa832a92b4c148dc69e3b88a8cc8480d4d...,"[{""ticker"": ""AMD"", ""sentiment"": ""positive"", ""s...","[""stock market"", ""inflation data"", ""banking st...",2026-01-13 22:07:09,"[""AMD"", ""INTC"", ""AMJB"", ""JPM"", ""JPMpC"", ""JPMpD...","Stock Market Today, Jan. 13: AMD Rallies as Ba...",The Motley Fool,AAPL
...,...,...,...,...,...,...,...,...,...,...,...
4,https://www.fool.com/investing/2026/01/05/shou...,Selena Maranjian,"The Magnificent Seven stocks (Apple, Amazon, A...",b245e74f9992361dd2b54c8097113f35bc0c8c30026f0f...,"[{""ticker"": ""AAPL"", ""sentiment"": ""neutral"", ""s...","[""Magnificent Seven"", ""S&P 500"", ""concentratio...",2026-01-05 20:15:00,"[""AAPL"", ""AMZN"", ""GOOG"", ""GOOGL"", ""META"", ""MSF...","Should Investors Be Worried That the ""Magnific...",The Motley Fool,AAPL
3,https://www.fool.com/investing/2026/01/05/3-bo...,Keithen Drury,Analyst predicts Nvidia will outperform its Ma...,b5bb4cbe57189d9d0b3b4bfb010923fb3328c421e05fb9...,"[{""ticker"": ""NVDA"", ""sentiment"": ""positive"", ""...","[""Nvidia"", ""AI buildout"", ""Rubin architecture""...",2026-01-05 18:15:00,"[""NVDA"", ""GOOG"", ""GOOGL"", ""MSFT"", ""AMZN"", ""MET...",3 Bold Nvidia Predictions For 2026,The Motley Fool,AAPL
2,https://www.fool.com/investing/2026/01/05/shou...,Bram Berkowitz,The article examines Warren Buffett's 1957 inv...,a37c713912b68263d24536092588e1cfa3fbef20adb0e5...,"[{""ticker"": ""AAPL"", ""sentiment"": ""positive"", ""...","[""Warren Buffett"", ""70/30 rule"", ""portfolio al...",2026-01-05 16:30:00,"[""AAPL""]",Should Investors Stick to Warren Buffett's 70/...,The Motley Fool,AAPL
1,https://www.fool.com/investing/2026/01/05/2-tr...,Prosper Junior Bakiny,"Amazon and Apple, despite underperforming in 2...",569acd46e35f725d3d65e8727affda0fcd63fb38f8c3bb...,"[{""ticker"": ""AMZN"", ""sentiment"": ""positive"", ""...","[""cloud computing"", ""AI services"", ""iPhone 17""...",2026-01-05 12:35:00,"[""AMZN"", ""AAPL"", ""GOOG"", ""GOOGL"", ""NVDA""]",2 Trillion-Dollar Stocks That Could Beat the M...,The Motley Fool,AAPL


In [11]:
LLM_news = prepare_news_for_llm(nvdia_news.results)

In [12]:
template = p.news_sentiment_template

prompt = template.invoke({"llm_news": json.dumps(LLM_news, indent=2), "ticker": ticker})

In [13]:
print(prompt)

messages=[SystemMessage(content='You are a financial analyst. You are given a list of news articles and a ticker. You need to analyze the news and provide sentiment analysis.\n                    \n                    You must respond in JSON format with this exact structure:\n                    {\n                        "short_term_sentiment": {\n                            "short_term_sentiment": "bullish/bearish/neutral",\n                            "short_term_confidence": 0.0-1.0,\n                            "short_term_reasoning": "explanation"\n                        },\n                        "long_term_sentiment": {\n                            "long_term_sentiment": "bullish/bearish/neutral",\n                            "long_term_confidence": 0.0-1.0,\n                            "long_term_reasoning": "explanation"\n                        },\n                        "key_events_outlook": [\n                            {\n                                "event": "eve

In [14]:
model_name = "qwen3:8b"  # Options: llama3.2, llama3.1, mistral, qwen2.5
model_provider = "Ollama"
pydantic_model = NewsSentiment
max_retries = 3

In [15]:
sentiment_result = call_llm(prompt, model_name, model_provider, pydantic_model, max_retries=max_retries)

In [16]:
pprint(sentiment_result)

NewsSentiment(
    short_term_sentiment=ShortTermSentiment(
        short_term_sentiment='bullish',
        short_term_confidence=0.85,
        short_term_reasoning=(
            "Tech stocks like NVIDIA, Apple, and AMD showed strong gains, while Intel's 146% surge over six months ind"
            "icates positive momentum. Warren Buffett's endorsement of index funds adds confidence, though banking sec"
            'tor declines suggest mixed sentiment.'
        ),
    ),
    long_term_sentiment=LongTermSentiment(
        long_term_sentiment='neutral',
        long_term_confidence=0.6,
        long_term_reasoning=(
            "While tech sector growth and Buffett's long-term strategies suggest potential, the banking sector's decli"
            'ne and regulatory uncertainties may temper long-term optimism. ETF comparisons highlight mixed performanc'
            'e across asset classes.'
        ),
    ),
    key_events_outlook=[
        KeyEventOutlook(
            event="Intel's Gove

In [17]:
template = p.news_sentiment_by_date_template

In [18]:
sentiment_results_by_date = []
for date, news in parse_news_by_date(nvdia_news.results).items():
    llm_news = prepare_news_for_llm(news)
    prompt_news_by_date = template.invoke({"llm_news": json.dumps(LLM_news, indent=2), "ticker": ticker, "date": date} )
    print(f'\nProcessing news on {date}: {len(news)} news')
    sentiment_result_by_date = call_llm(prompt_news_by_date, model_name, model_provider, NewsSentimentByDate, max_retries=max_retries)
    sentiment_results_by_date.append(sentiment_result_by_date)

10 days of news parsed

Processing news on 2026-01-05: 7 news

Processing news on 2026-01-06: 11 news

Processing news on 2026-01-07: 10 news

Processing news on 2026-01-08: 11 news

Processing news on 2026-01-09: 9 news

Processing news on 2026-01-10: 10 news

Processing news on 2026-01-11: 3 news

Processing news on 2026-01-12: 7 news

Processing news on 2026-01-13: 8 news

Processing news on 2026-01-14: 4 news


In [19]:
for result in sentiment_results_by_date:
    pprint(result)

NewsSentimentByDate(
    summary=(
        "The news articles from 2026-01-05 cover a range of topics, including stock market movements (e.g., Intel's 8%"
        " surge due to AI investments, banking sector struggles, and Buffett's endorsement of index funds), ETF compar"
        'isons (VOOG vs RSP), and updates on tech companies like NVDA and AAPL. Key themes include optimism about AI-d'
        'riven growth, caution in banking, and strategic ETF recommendations.'
    ),
    sentiment='Mixed but leaning positive',
    reasoning=(
        "The sentiment is positive for tech stocks (e.g., Intel, NVDA) due to AI-driven growth and Buffett's endorseme"
        "nt of index funds, which boosts investor confidence. However, the banking sector's struggles and mixed ETF pe"
        'rformance introduce caution. Overall, the news suggests a bullish market for tech but warns of sector-specifi'
        'c risks.'
    ),
    date='2026-01-05',
    stock_price_movement=(
        'Yes, the news 